<a href="https://colab.research.google.com/github/akash007data/akashl-learning-git/blob/main/Assignment2_G24AI2040_AKASH_TIWARI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!sudo apt-get update -y
!sudo apt-get install openjdk-11-jdk-headless -y


Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,152 kB]
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,269 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Pac

In [ ]:
!wget https://archive.apache.org/dist/hadoop/common/hadoop-3.3.6/hadoop-3.3.6.tar.gz


--2025-07-28 17:30:35--  https://archive.apache.org/dist/hadoop/common/hadoop-3.3.6/hadoop-3.3.6.tar.gz
Resolving archive.apache.org (archive.apache.org)... 65.108.204.189, 2a01:4f9:1a:a084::2
Connecting to archive.apache.org (archive.apache.org)|65.108.204.189|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 730107476 (696M) [application/x-gzip]
Saving to: ‘hadoop-3.3.6.tar.gz’

hadoop-3.3.6.tar.gz 100%[===================>] 696.28M  18.3MB/s    in 40s     

2025-07-28 17:31:16 (17.3 MB/s) - ‘hadoop-3.3.6.tar.gz’ saved [730107476/730107476]



In [ ]:
!tar -xzf hadoop-3.3.6.tar.gz
!mv hadoop-3.3.6 /usr/local/hadoop


In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["HADOOP_HOME"] = "/usr/local/hadoop"
os.environ["PATH"] += ":/usr/local/hadoop/bin"


In [ ]:
!pip install mrjob

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.6/439.6 kB 9.6 MB/s eta 0:00:00


In [11]:
# downloading the csv files directly from assignment link

# cruise ships data
!wget https://raw.githubusercontent.com/TakMashhido/PGD-BigData-Tutorial/refs/heads/main/Dataset/cruise.csv

# customer churn data
!wget https://raw.githubusercontent.com/TakMashhido/PGD-BigData-Tutorial/refs/heads/main/Dataset/customer_churn.csv

# e-commerce customer data
!wget https://raw.githubusercontent.com/TakMashhido/PGD-BigData-Tutorial/refs/heads/main/Dataset/e-com_customer.csv


--2025-07-28 17:49:25--  https://raw.githubusercontent.com/TakMashhido/PGD-BigData-Tutorial/refs/heads/main/Dataset/cruise.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8734 (8.5K) [text/plain]
Saving to: ‘cruise.csv’

cruise.csv          100%[===================>]   8.53K  --.-KB/s    in 0s      

2025-07-28 17:49:25 (52.4 MB/s) - ‘cruise.csv’ saved [8734/8734]

--2025-07-28 17:49:25--  https://raw.githubusercontent.com/TakMashhido/PGD-BigData-Tutorial/refs/heads/main/Dataset/customer_churn.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response

## Q1 - Cruise Line Aggregations

### What I do:
- I take cruise.csv file and read line by line
- From each line I get cruise line name, tonnage and crew size
- I give to reducer how many ship, how much tonnage and crew
- In reducer I make average tonnage, and max crew
- final output is ship count, avg tonnage, and max crew


In [13]:
# write the mrjob class to a file called cruise_agg.py
code = '''
from mrjob.job import MRJob
from mrjob.step import MRStep

class CruiseLineAgg(MRJob):

    def mapper(self, _, line):
        if "Cruise_line" in line:
            return
        try:
            part = line.split(",")
            cruise_line = part[1]
            tonnage = float(part[3])
            crew = float(part[8])
            yield cruise_line, (1, tonnage, crew)
        except:
            pass

    def combiner(self, cruise_line, values):
        cnt = 0
        ton_sum = 0
        max_crew = 0
        for v in values:
            cnt += v[0]
            ton_sum += v[1]
            max_crew = max(max_crew, v[2])
        yield cruise_line, (cnt, ton_sum, max_crew)

    def reducer(self, cruise_line, values):
        cnt = 0
        ton_sum = 0
        max_crew = 0
        for v in values:
            cnt += v[0]
            ton_sum += v[1]
            max_crew = max(max_crew, v[2])
        avg_ton = round(ton_sum / cnt, 2)
        yield cruise_line, (cnt, avg_ton, max_crew)

if __name__ == "__main__":
    CruiseLineAgg.run()
'''

# save to file
with open("cruise_agg.py", "w") as f:
    f.write(code)


In [15]:
!ls

cruise_agg.py  customer_churn.csv  hadoop-3.3.6.tar.gz
cruise.csv     e-com_customer.csv  sample_data


In [16]:
!python3 cruise_agg.py cruise.csv


No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/cruise_agg.root.20250728.175502.034288
Running step 1 of 1...
job output is in /tmp/cruise_agg.root.20250728.175502.034288/output
Streaming final output from /tmp/cruise_agg.root.20250728.175502.034288/output...
"Oceania"	[3, 30.28, 4.0]
"Orient"	[1, 22.08, 3.5]
"P&O"	[6, 77.86, 12.2]
"Princess"	[17, 87.54, 12.38]
"Regent_Seven_Seas"	[5, 32.14, 4.47]
"Royal_Caribbean"	[23, 107.01, 21.0]
"Seabourn"	[3, 10.0, 1.6]
"Silversea"	[4, 20.9, 2.95]
"Star"	[6, 30.77, 12.0]
"Windstar"	[3, 8.48, 1.8]
"Azamara"	[2, 30.28, 3.55]
"Carnival"	[22, 84.65, 19.1]
"Celebrity"	[10, 76.16, 9.99]
"Costa"	[11, 71.1, 10.9]
"Crystal"	[2, 59.5, 6.36]
"Cunard"	[3, 103.91, 12.53]
"Disney"	[2, 83.17, 9.45]
"Holland_American"	[14, 60.5, 8.42]
"MSC"	[8, 63.77, 13.13]
"Norwegian"	[13, 63.72, 13.0]
Removing temp directory /tmp/cruise_agg.root.20250728.175502.034288...


In [21]:
from cruise_agg import CruiseLineAgg
from io import BytesIO

# Read cruise.csv and encode it in bytes
with open("cruise.csv", "r") as f:
    cruise_data = f.read().encode("utf-8")

# Create job runner
job = CruiseLineAgg(args=[])
runner = job.make_runner()
runner._stdin = BytesIO(cruise_data)

# Run the job
runner.run()

# Output
output = list(job.parse_output(runner.cat_output()))
for cruise_line, values in output:
    print(f"{cruise_line} -> Ships: {values[0]}, Avg Tonnage: {values[1]}, Max Crew: {values[2]}")


Oceania -> Ships: 3, Avg Tonnage: 30.28, Max Crew: 4.0
Orient -> Ships: 1, Avg Tonnage: 22.08, Max Crew: 3.5
P&O -> Ships: 6, Avg Tonnage: 77.86, Max Crew: 12.2
Princess -> Ships: 17, Avg Tonnage: 87.54, Max Crew: 12.38
Regent_Seven_Seas -> Ships: 5, Avg Tonnage: 32.14, Max Crew: 4.47
Royal_Caribbean -> Ships: 23, Avg Tonnage: 107.01, Max Crew: 21.0
Seabourn -> Ships: 3, Avg Tonnage: 10.0, Max Crew: 1.6
Silversea -> Ships: 4, Avg Tonnage: 20.9, Max Crew: 2.95
Star -> Ships: 6, Avg Tonnage: 30.77, Max Crew: 12.0
Windstar -> Ships: 3, Avg Tonnage: 8.48, Max Crew: 1.8
Azamara -> Ships: 2, Avg Tonnage: 30.28, Max Crew: 3.55
Carnival -> Ships: 22, Avg Tonnage: 84.65, Max Crew: 19.1
Celebrity -> Ships: 10, Avg Tonnage: 76.16, Max Crew: 9.99
Costa -> Ships: 11, Avg Tonnage: 71.1, Max Crew: 10.9
Crystal -> Ships: 2, Avg Tonnage: 59.5, Max Crew: 6.36
Cunard -> Ships: 3, Avg Tonnage: 103.91, Max Crew: 12.53
Disney -> Ships: 2, Avg Tonnage: 83.17, Max Crew: 9.45
Holland_American -> Ships: 14, Avg

## Q2 - Company Churn Rate

- we use file called customer_churn.csv  
- inside it we check Churn == 1 and also count all customer  
- we group by Company name  
- then we make churn rate = Churned / Total  
- also, we use small file vip_companies.txt to select only VIP company  
- it’s like 2 step mrjob job  


## Q2 - Company Churn Rate

### What I do:
- first I emit 1 for total and 1 for churned if churn == 1
- second step I add them and do churned / total
- I only keep VIP company which are in small txt file
- I make final result to 4 decimal point


In [23]:
# small file with vip company names
vip_list = """Alpha Inc
Beta Corp
Zeta Ltd
"""

with open("vip_companies.txt", "w") as f:
    f.write(vip_list)


In [34]:
# making file for Q2 churn rate thing
code = '''
from mrjob.job import MRJob
from mrjob.step import MRStep

class ChurnRate(MRJob):

    def configure_args(self):
        super(ChurnRate, self).configure_args()
        self.add_file_arg('--vip')

    def load_vip(self):
        self.vip_set = set()
        with open(self.options.vip, 'r') as f:
            for line in f:
                self.vip_set.add(line.strip())

    def steps(self):
        return [
            MRStep(mapper_init=self.load_vip, mapper=self.mapper_step1, reducer=self.reducer_step1),
            MRStep(reducer=self.reducer_step2)
        ]

    def mapper_step1(self, _, line):
        if "Company" in line:
            return
        try:
            part = line.strip().split(",")
            company = part[8]
            churn = int(part[-1])
            if company in self.vip_set:
                yield company, ("TOTAL", 1)
                if churn == 1:
                    yield company, ("CHURNED", 1)
        except:
            pass

    def reducer_step1(self, company, values):
        total = 0
        churned = 0
        for tag, count in values:
            if tag == "TOTAL":
                total += count
            elif tag == "CHURNED":
                churned += count
        yield company, (churned, total)

    def reducer_step2(self, company, values):
        for churned, total in values:
            if total > 0:
                rate = round(churned / total, 4)
                yield company, rate

if __name__ == '__main__':
    ChurnRate.run()
'''

# write to file
with open("churn_rate.py", "w") as f:
    f.write(code)


In [38]:
# making small customer churn file for testing
test_csv = """Company,Location,Churn
Alpha Inc,Delhi,1
Alpha Inc,Delhi,0
Beta Corp,Mumbai,1
Beta Corp,Mumbai,1
Beta Corp,Mumbai,0
Gamma LLC,Chennai,1
Zeta Ltd,Pune,0
Zeta Ltd,Pune,0
Zeta Ltd,Pune,1
"""

with open("mini_customer.csv", "w") as f:
    f.write(test_csv)


In [41]:
import csv

with open("customer_churn.csv", newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for i, row in enumerate(reader):
        if i >= 10:
            break
        print(row["Company"])


Harvey LLC
Wilson PLC
Miller, Johnson and Wallace
Smith Inc
Love-Jones
Kelly-Warren
Reynolds-Sheppard
Singh-Cole
Lopez PLC
Reed-Martinez


In [44]:
vip_list = """Harvey LLC
Wilson PLC
Miller, Johnson and Wallace
Smith Inc
"""

with open("vip_companies.txt", "w") as f:
    f.write(vip_list)


In [47]:
!python3 churn_rate.py customer_churn.csv --vip vip_companies.txt


No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/churn_rate.root.20250728.181247.781056
Running step 1 of 2...
Running step 2 of 2...
job output is in /tmp/churn_rate.root.20250728.181247.781056/output
Streaming final output from /tmp/churn_rate.root.20250728.181247.781056/output...
"Wilson PLC"	0.0
Removing temp directory /tmp/churn_rate.root.20250728.181247.781056...


In [52]:
from mrjob.util import to_lines
from churn_rate import ChurnRate

print("🙃 Churn rates for VIP companies in clean style:\n")

mr_job = ChurnRate(args=["customer_churn.csv", "--vip", "vip_companies.txt"])

with mr_job.make_runner() as runner:
    runner.run()
    for line in to_lines(runner.cat_output()):
        line = line.decode("utf-8").strip()
        if '\t' in line:
            company, rate = line.split('\t')
            print(f"{company} -> Churn Rate: {float(rate):.4f}")


🙃 Churn rates for VIP companies in clean style:

"Wilson PLC" -> Churn Rate: 0.0000


######Q3 - State-wise Spending
What I do:
I read the CSV and take state code from the Address column (last 2 letters before closing ").

I collect Yearly Amount Spent for each state.

I then sum total spending per state.

At the end, I print top 5 states with highest total spending.

It’s a 2-step mrjob job:
1st: collect and emit (state, amount)
2nd: sum and sort descending

In [53]:
# make mini e-commerce file to test our map reduce
mini_ecom = """Email,Address,Avatar,Avg Session Length,Time on App,Time on Website,Length of Membership,Yearly Amount Spent
a@a.com,"100 Lane Road, CA 90001",Red,33.0,12.0,37.0,4.0,600.00
b@b.com,"200 Fast Street, TX 75001",Blue,34.0,11.0,36.0,3.0,550.00
c@c.com,"300 Slow Ave, TX 75001",Green,32.0,12.5,35.0,2.5,620.00
d@d.com,"400 Jump Blvd, CA 90001",Yellow,33.5,12.2,38.0,4.5,650.00
e@e.com,"500 Rest Lane, NY 10001",Purple,31.0,10.5,34.5,2.0,500.00
f@f.com,"600 Lake View, TX 75001",Black,35.0,11.5,36.5,3.5,580.00
g@g.com,"700 Park St, WA 98101",White,34.5,12.8,37.5,4.2,700.00
"""

# write mini test file
with open("mini_ecom.csv", "w") as f:
    f.write(mini_ecom)


In [54]:
%%writefile state_spend.py
from mrjob.job import MRJob
from mrjob.step import MRStep
import csv

class StateSpend(MRJob):

    def steps(self):
        return [
            MRStep(mapper=self.mapper_get_state_spend,
                   reducer=self.reducer_sum_state),
            MRStep(reducer=self.reducer_top5)
        ]

    def mapper_get_state_spend(self, _, line):
        # use csv reader to split line safely
        try:
            row = next(csv.reader([line]))
            if row[0] == "Email":  # skip header
                return
            address = row[1]
            spent = float(row[-1])
            # get state from address (after comma)
            state = address.split(",")[1].strip()[:2]
            yield state, spent
        except:
            pass  # if anything goes wrong, skip line

    def reducer_sum_state(self, state, amounts):
        # add all amount for state
        yield None, (sum(amounts), state)

    def reducer_top5(self, _, state_amounts):
        # sort all state by spending, print top 5
        top5 = sorted(state_amounts, reverse=True)[:5]
        for amount, state in top5:
            yield state, round(amount, 2)

if __name__ == "__main__":
    StateSpend.run()


Writing state_spend.py


In [55]:
!python3 state_spend.py mini_ecom.csv


No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/state_spend.root.20250728.181958.805356
Running step 1 of 2...
Running step 2 of 2...
job output is in /tmp/state_spend.root.20250728.181958.805356/output
Streaming final output from /tmp/state_spend.root.20250728.181958.805356/output...
"TX"	1750.0
"CA"	1250.0
"WA"	700.0
"NY"	500.0
Removing temp directory /tmp/state_spend.root.20250728.181958.805356...


In [56]:
!python3 state_spend.py e-com_customer.csv


No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/state_spend.root.20250728.182021.126038
Running step 1 of 2...
Running step 2 of 2...
job output is in /tmp/state_spend.root.20250728.182021.126038/output
Streaming final output from /tmp/state_spend.root.20250728.182021.126038/output...
"Bo"	7546.97
"SC"	6820.3
"DE"	6644.98
"MO"	6402.57
"VT"	6150.03
Removing temp directory /tmp/state_spend.root.20250728.182021.126038...


In [59]:
from state_spend import StateSpend
from mrjob.util import to_lines

print("🧾 Final Clean Top 5 States:\n")

mr_job = StateSpend(args=["e-com_customer.csv"])

with mr_job.make_runner() as runner:
    runner.run()
    for line in to_lines(runner.cat_output()):
        line = line.decode("utf-8").strip()  # convert bytes to string
        parts = line.split("\t")
        state = parts[0].strip('"')
        total = float(parts[1])
        print(f"{state} -> ₹{total:.2f}")


🧾 Final Clean Top 5 States:

Bo -> ₹7546.97
SC -> ₹6820.30
DE -> ₹6644.98
MO -> ₹6402.57
VT -> ₹6150.03


###Q4 - Two-step Ship Filter & Median Length
What I do:
I use cruise.csv file and first step, I check if passenger_density > 35.

I emit (Cruise_line, length) only for those ships.

In second step, I group by Cruise_line and compute median of lengths.

If number of ships is even, I take average of middle two.

Final result shows median length per cruise line, rounded to 2 decimal point.

In [60]:
mini_cruise = """Ship_name,Cruise_line,Age,Tonnage,passengers,length,cabins,passenger_density,crew
Journey,Azamara,6,30.28,6.94,5.94,3.55,42.64,3.55
Quest,Azamara,6,30.28,6.94,6.14,3.55,42.64,3.55
Conquest,Carnival,11,110.0,29.74,9.53,14.88,36.99,19.1
Destiny,Carnival,17,101.35,26.42,8.92,13.21,38.36,10.0
Fantasy,Carnival,23,70.37,20.56,8.55,10.22,34.23,9.2
Elation,Carnival,15,70.37,20.52,8.55,10.2,34.29,9.2
"""

with open("mini_cruise_median.csv", "w") as f:
    f.write(mini_cruise)


In [63]:
from mrjob.job import MRJob
from mrjob.step import MRStep

class ShipMedianLength(MRJob):

    def steps(self):
        return [
            MRStep(mapper=self.mapper_filter,
                   reducer=self.reducer_group_lengths),
            MRStep(mapper=self.mapper_identity,
                   reducer=self.reducer_median)
        ]

    def mapper_filter(self, _, line):
        if line.startswith("Ship_name"):  # skip header
            return
        parts = line.strip().split(",")
        try:
            cruise_line = parts[1]
            length = float(parts[5])
            density = float(parts[7])
            if density > 35.0:
                yield cruise_line, length
        except:
            pass  # in case of bad row

    def reducer_group_lengths(self, cruise_line, lengths):
        # collect all lengths in a list
        yield cruise_line, list(lengths)

    def mapper_identity(self, cruise_line, lengths_list):
        yield cruise_line, lengths_list

    def reducer_median(self, cruise_line, lengths_lists):
        all_lengths = []
        for lst in lengths_lists:
            all_lengths.extend(lst)
        all_lengths.sort()
        n = len(all_lengths)
        if n == 0:
            return
        if n % 2 == 1:
            median = all_lengths[n // 2]
        else:
            median = (all_lengths[n // 2 - 1] + all_lengths[n // 2]) / 2.0
        yield cruise_line, round(median, 2)

if __name__ == '__main__':
    ShipMedianLength.run()


usage: colab_kernel_launcher.py [options] [input files]
colab_kernel_launcher.py: error: unrecognized arguments: -f


SystemExit: 2

In [62]:
test_data = """Ship_name,Cruise_line,Age,Tonnage,passengers,length,cabins,passenger_density,crew
ShipA,Alpha,10,70,20,7.5,10,35.1,9.0
ShipB,Alpha,12,72,22,8.0,11,36.0,9.5
ShipC,Beta,8,50,18,6.0,8,33.0,6.0
ShipD,Beta,6,60,25,7.0,10,37.5,7.5
ShipE,Beta,7,65,23,7.2,10,36.2,7.8
ShipF,Gamma,5,40,15,5.5,6,34.0,5.0
"""

with open("mini_cruise_median.csv", "w") as f:
    f.write(test_data)

print("✅ mini_cruise_median.csv created.")


✅ mini_cruise_median.csv created.


In [64]:
!python3 median_length.py mini_cruise_median.csv


python3: can't open file '/content/median_length.py': [Errno 2] No such file or directory


In [65]:
code = '''from mrjob.job import MRJob
from mrjob.step import MRStep
import statistics
import csv

class MedianShipLength(MRJob):

    def steps(self):
        return [
            MRStep(mapper=self.mapper_filter,
                   reducer=self.reducer_collect_lengths),
            MRStep(reducer=self.reducer_find_median)
        ]

    def mapper_filter(self, _, line):
        # skip header
        if "Ship_name" in line:
            return
        try:
            row = list(csv.reader([line]))[0]
            cruise_line = row[1]
            passenger_density = float(row[7])
            length = float(row[5])
            if passenger_density > 35.0:
                yield cruise_line, length
        except:
            pass  # skip bad lines

    def reducer_collect_lengths(self, cruise_line, lengths):
        yield cruise_line, list(lengths)

    def reducer_find_median(self, cruise_line, lengths_lists):
        all_lengths = []
        for l in lengths_lists:
            all_lengths.extend(l)
        median_val = statistics.median(all_lengths)
        yield cruise_line, round(median_val, 2)

if __name__ == "__main__":
    MedianShipLength.run()
'''

with open("median_length.py", "w") as f:
    f.write(code)

print("✅ File 'median_length.py' created.")


✅ File 'median_length.py' created.


In [66]:
!python3 median_length.py mini_cruise_median.csv


No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/median_length.root.20250728.182451.073560
Running step 1 of 2...
Running step 2 of 2...
job output is in /tmp/median_length.root.20250728.182451.073560/output
Streaming final output from /tmp/median_length.root.20250728.182451.073560/output...
"Alpha"	7.75
"Beta"	7.1
Removing temp directory /tmp/median_length.root.20250728.182451.073560...


In [67]:
!python3 median_length.py cruise.csv


No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/median_length.root.20250728.182512.346025
Running step 1 of 2...
Running step 2 of 2...
job output is in /tmp/median_length.root.20250728.182512.346025/output
Streaming final output from /tmp/median_length.root.20250728.182512.346025/output...
"Oceania"	5.94
"P&O"	8.56
"Princess"	9.51
"Regent_Seven_Seas"	6.15
"Royal_Caribbean"	10.2
"Seabourn"	4.4
"Silversea"	5.55
"Star"	2.8
"Windstar"	6.17
"Azamara"	5.94
"Carnival"	9.52
"Celebrity"	9.65
"Costa"	8.28
"Crystal"	7.86
"Cunard"	9.64
"Disney"	9.64
"Holland_American"	7.77
"MSC"	8.23
"Norwegian"	9.0
Removing temp directory /tmp/median_length.root.20250728.182512.346025...
